### Что было сделано

- Реализовано решение Ф-П методом замены переменной на Python
- Извлечены статистики распределений для каждого момента времени
- Набор статистик за несколько последних периодов + текущая позиция = состояние среды
- Вознаграждение = позиция * points - commission
- Реализован метод обучения с подкреплением Actor-Critic
    - Value function approximation - линейная функция от вектора состояния среды
    - Policy function approximation - линейная функция от вектора состояния среды


### Решение уравнения Ф-П методом замены переменной

### Обучение с подкреплением

Обучением с подкреплением называется группа методов обучения, в которых обучение происходит посредством взаимодействия с некоторой средой. Обучаемый элемент, принимающий решения для достижения определенной цели называется агентом. Элемент, с которым происходит взаимодействия агента, представляет собой внешнюю среду. Агент выбирает действия, среда отвечает на эти действия и переходит в новое состояние, наблюдаемое агентом. Среда также отвечает агенту числовым вознаграждением, при этом основной задачей обучения агента является максимизация сумарного полученного вознаграждения.

Для определения задачи обучения с подкреплением необходимо определить спецификацию среды и правило формирования вознаграждения.

Формально агент взаимодействует со средой в дискретные моменты времени $t = 0, 1, 2, ...$. На каждом шаге $t$ агент наблюдает состояние внешней среды $S_t \in \mathcal{S}$, где $\mathcal{S}$ множество всех возможных состояний, и на его основе выбирает действие $A_t \in \mathcal{A}(S_t)$, где $\mathcal{A}(S_t)$ множество действий, доступных в состоянии $S_t$, в результате действия в следующий момент времени агент получает вознаграждение от среды, $R_{t+1} \in \mathcal{R}$ и наблюдает новое состояние $S_{t+1}$.

В каждый момент времени для заданного состояния агент определяет вероятности выбора каждого действия из множества доступных. Это определяет стратегию агента (или политику), она обозначается $\pi_t$, где $\pi_t(a|s)$ - это вероятность, что $A_t = a$ при условии $S_t = s$. Методы обучения с подкреплением определяют как агент меняет свою стратегию действий в результате накопленного опыта взаимодействия со средой.

Для задач, в которых нет терминальных состояний и агент взаимодействует со средой бесконечно долго или неопределенное время в роли целевого вознаграждения используется не простая сумма всех полученных вознаграждений, а дисконтированная. Таким образом агент выбирает такое действие $A_t$^не которое максимизирует ожидаемую сумму дисконтированных вознаграждений:

$$G_t = R_{t+1} + \gamma R_{t+2} + \gamma^{2} R_{3} + ... = \displaystyle\sum_{k=0}^{\infty} \gamma^{k} R_{t+k+1}$$

где $0 \le \gamma \le 1$ является параметром и называется дисконтирующим множетелем.


### Марковский процесс принятия решения

Если состояние среды обладает марковым свойством, то есть для всех $r, s', s$ и $a$ выполняется следующее:

$$p(s',r \mid s,a) = Pr \{ S_{t+1} = s', R_{t+1} = r \mid S_t = s, A_t = a \} $$

то говорят, что вся задача обучения с подкреплением обладает свойством Маркова. В этом случае задача обучения с подкреплением называется марковским процессом принятия решения (МППР), а в случае конечности пространства состояний и действий называется конечным марковским процессом принятия решений.

Основным подходом почти всех методов обучения с подкреплением является оценка специальной функции ценности состояния (или пары состояние-действие), которая определяет ценность нахождения агента в определенном состоянии (или ценность действия в заданном состоянии). Понятие ценности задается с точки зрения ожидаемого сумарного вознаграждения. Вознаграждение же зависит от тех действий, которые агент будет делать. Таким образом функция ценности задается с учетом стратегии действий:

$$
v_{\pi}(s) = \mathbf{E}_{\pi} \left[ G_t \mid S_t = s \right] \\
= \mathbf{E}_{\pi} \left[ \displaystyle\sum_{k=0}^{\infty} \gamma^{k} R_{t+k+1} \mid S_t = s \right] \\
= \mathbf{E}_{\pi} \left[ R_{t+1} + \gamma \displaystyle\sum_{k=0}^{\infty} \gamma^{k} R_{t+k+2} \mid S_t = s \right] \\
= \sum_{a} \pi(a \mid s) \sum_{s'} \sum_{r} p(s',r \mid s,a) \left[ r + \gamma \mathbf{E}_{\pi} \left[ \displaystyle\sum_{k=0}^{\infty} \gamma^{k} R_{t+k+2} \mid S_t = s' \right] \right] \\
= \sum_{a} \pi(a \mid s) \sum_{s', r} p(s',r \mid s,a) \left[ r + \gamma v_{\pi}(s') \right]
$$

В результате получили уравнение Белмана для функции ценности состояния $v_{\pi}(s)$

Функцию ценности действия для заданного состояния можно определить следующим образом:

$$q_{\pi}(s, a) = \mathbf{E}_{\pi} \left[ G_t \mid S_t = s, A_t = a \right] = \mathbf{E}_{\pi} \left[ \displaystyle\sum_{k=0}^{\infty} \gamma^{k} R_{t+k+1} \mid S_t = s, A_t = a \right]$$

Аналогично для $q_{\pi}(s, a)$ можно получить уравнение Белмана:

$$q_{\pi}(s, a) = = \sum_{a} \pi(a \mid s) \sum_{s', r} p(s',r \mid s,a) \left[ r + \gamma q_{\pi}(s', a) \right]
$$

Лучшая стратегия предполагает наличие оптимальных функций ценности состояния $v_{*}(s) = \max_{\pi} v_{\pi}(s)$ и ценности действия в заданном состоянии $q_{*}(s, a) = \max_{\pi} q_{\pi}(s, a)$. Если эти функции заданы, то выбор оптимального действия осуществляется очевидно, соответственно может быть определена оптимальная политика. Для задач с небольшими наборами состояний и действий функции ценности и политики могут быть представлены в виде таблиц. В других же случаях часто используются компактные параметрические представления, в роли которых могут использоваться линейные функции, нейронные сети, деревья принятия решения и другие.

### Постановка задачи обучения с подкреплением для торговой системы

#### Представление состояния среды

### Алгоритм Actor-Critic
